# Gradio APP to help generate prompt

In [1]:
import gradio as gr
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

openai = OpenAI()



In [7]:
system_prompt = """
You are PromptSmith, an expert assistant specialized in designing high-quality prompts for other AI assistants.

Your Role:

Take the user’s input (their goal, use case, or idea) and transform it into a clear, effective, and optimized prompt.

When appropriate, generate both a System Prompt (to define the assistant’s role and behavior) and a User Prompt (to give specific instructions).

Always ensure your prompts are copy-paste ready.

Your Process:

Clarify Objective: Identify what the user wants the target assistant to do.

Refine Requirements: Ask follow-up questions if needed (tone, style, format, audience, constraints).

Craft Prompt(s): Write precise, unambiguous instructions that guide the assistant toward the user’s goal.

Optimize for Use:

Actionable → the assistant knows exactly what to do.

Contextual → incorporates user-provided details.

Flexible → easy for the user to adapt.

Present Clearly: Output the final result in a structured way, labeling each part as “System Prompt” or “User Prompt.”

Output Format:

Always provide the final answer like this:

System Prompt:
[well-crafted system prompt text]

User Prompt:
[well-crafted user prompt text or template for the user to fill in]
"""

def create_user_prompt(user_input):
    user_prompt = f"""
    {user_input}
    """
    return user_prompt

In [16]:
def user_prompt_for_prompt_engineering(history):
    
    messages = [
        {"role": "system", "content": system_prompt}
    ]+history

    result = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True
    )
    print("helloooo")
    reply = ""
    for chunk in result:
        reply += chunk.choices[0].delta.content or ""
        yield history + [{"role":"assistant", "content":reply}]


In [13]:
def on_submit(message, history):
    print(history)
    print(message)
    if history is None:
        history = []
    history += [{'role': "user", "content": message}]
    return "", history

In [ ]:
with gr.Blocks() as ui:
    gr.Markdown("## Prompt Generation Assistant")
    with gr.Row():
        with gr.Column():
            user_input = gr.Textbox(label="Your input:", placeholder="Enter the idea of the prompt that you want to generate...", lines=10)
            submit_button = gr.Button("Submit")
        with gr.Column():
            output = gr.Chatbot(label="Output", type="messages")

    submit_button.click(fn=on_submit, inputs=[user_input, output], outputs=[user_input, output]).then(user_prompt_for_prompt_engineering, inputs=[output], outputs=[output])
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7908
* To create a public link, set `share=True` in `launch()`.


[]
Hello
helloooo
[{'role': 'user', 'metadata': None, 'content': 'Hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hello! How can I assist you today? Please share your goal or idea, and I'll help create a prompt for you.", 'options': None}]
Can you help me create the perfect prompt for an assistant that will properly document python code ? It should add the right amount of comments and docstrings to the code.
helloooo
[{'role': 'user', 'metadata': None, 'content': 'Hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Hello! How can I assist you today? Please share your goal or idea, and I'll help create a prompt for you.", 'options': None}, {'role': 'user', 'metadata': None, 'content': 'Can you help me create the perfect prompt for an assistant that will properly document python code ? It should add the right amount of comments and docstrings to the code.', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': "Absolu